In [ ]:
import networkx as nx
from pathlib import Path

base_path = Path("..", "..", "data", "graphs", "foodcom")

def load_nx_graph(path: Path) -> nx.Graph:
    return nx.read_graphml(path)


In [ ]:
B = load_nx_graph(base_path / "bipartite_recipe_ingredient.graphml")
nx.is_bipartite(B)

In [ ]:
from typing import List
from nltk.metrics.distance import edit_distance

def return_ingredient_given_query(ingredient_query: str, graph: nx.Graph, max_amount: int = 100) -> List[str]:
    """
    Returns a list with the ingredients that matches the query in the graph
    """ 
    ingredient_nodes: List[str] = [x for x in graph.nodes if graph.nodes[x]["type"] == "ingredient"]
    filtered_list = []
    
    # Filter
    for ingredient in ingredient_nodes:
        if ingredient_query in ingredient: # Contain filter
            filtered_list.append(ingredient)
    
    ranked_list = []

    # Ranking by Levenshtein distance 
    for ingredient in filtered_list:
        ranked_list.append((edit_distance(ingredient, ingredient_query), ingredient))
    
    # return [x for x in sorted(ranked_list)][:max_amount]
    return [x[1].removesuffix("_ingredient") for x in sorted(ranked_list)][:max_amount]

def return_recipe_given_query(recipe_query: str, graph: nx.Graph, max_amount: int = 100) -> List[str]:
    recipe_nodes: List[str] = [x for x in graph.nodes if graph.nodes[x]["type"] == "recipe"]
    filtered_list = []
    
    # Filter
    for recipe in recipe_nodes:
        if recipe_query in recipe: # Contain filter
            filtered_list.append(recipe)
    
    ranked_list = []

    # Ranking by Levenshtein distance 
    for recipe in filtered_list:
        ranked_list.append((edit_distance(recipe, recipe_query), recipe))
    
    # return [x for x in sorted(ranked_list)][:max_amount]
    return [x[1].removesuffix("_recipe") for x in sorted(ranked_list)][:max_amount]

def return_available_recipes_given_ingredients(ingredients_query: List[str], graph: nx.Graph, exact_ingredient_match: bool = False, max_amount: int = 100) -> List[str]:
    """
    Returns the recipes that can be made with any of the given ingredients.
    """
    ingredients_query = [x + "_ingredient" for x in ingredients_query]

    ingredient_nodes: List[str] = [x for x in graph.nodes if graph.nodes[x]["type"] == "ingredient"]
    ingredient_nodes = set(ingredient_nodes).intersection(ingredients_query)

    recipe_nodes = set(recipe for ingredient in ingredient_nodes for recipe in graph[ingredient])

    subgraph: nx.Graph = graph.subgraph(recipe_nodes.union(ingredient_nodes))

    # Ranking by degree
    recipe_nodes = [(len(list(subgraph.neighbors(recipe))), recipe) for recipe in subgraph.nodes if subgraph.nodes[recipe]["type"] == "recipe"]
    
    return [x[1].removesuffix("_recipe") for x in sorted(recipe_nodes, reverse=True) if not exact_ingredient_match or x[0] == len(ingredient_nodes)][:max_amount]
    # return [x for x in sorted(recipe_nodes, reverse=True)][:max_amount]

def return_ingredients_given_recipe(recipe_query: str, graph: nx.Graph) -> List[str]:
    return [x.removesuffix("_ingredient") for x in graph.neighbors(recipe_query + "_recipe")]


In [ ]:
ingredients = return_ingredient_given_query("egg", B)
print(ingredients)
recipes = return_available_recipes_given_ingredients(["egg", "flour", "pineapple"], B, True)
print(recipes)
recipe_ingredients = return_ingredients_given_recipe("zucchini pineapple wheat bread", B)
print(recipe_ingredients)
recipes = return_recipe_given_query("bread", B)
print(recipes)
